In [1]:
%%sh
wget -N https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
unzip -o bank-additional.zip

Archive:  bank-additional.zip
  inflating: bank-additional/bank-additional-names.txt  
  inflating: bank-additional/bank-additional.csv  
  inflating: bank-additional/bank-additional-full.csv  


--2021-09-25 21:26:00--  https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip
Resolving sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)... 52.218.235.41
Connecting to sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com (sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com)|52.218.235.41|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘bank-additional.zip’ not modified on server. Omitting download.



In [2]:
import sagemaker

print(sagemaker.__version__)

2.59.1


In [3]:
import boto3
region = boto3.Session().region_name

def resolve_sm_role():
    client = boto3.client('iam', region_name=region)
    response_roles = client.list_roles(
        PathPrefix='/',
    # Marker='string',
        MaxItems=999
    )
    for role in response_roles['Roles']:
        if role['RoleName'].startswith('AmazonSageMaker-ExecutionRole-'):
        #print('Resolved SageMaker IAM Role to: ' + str(role))
            return role['Arn']
        raise Exception('Could not resolve what should be the SageMaker role to be used')

    #resolve_sm_role()
    #role = get_execution_role()
    role = resolve_sm_role()
    return role 
role = resolve_sm_role()    

In [5]:
import pandas as pd
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
data[:5] # Show the first five lines

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [6]:
print(data.shape)

(41188, 21)


In [7]:
prefix = 'sagemaker/DEMO-smprocessing/input'

input_data = sagemaker.Session().upload_data(path='./bank-additional/bank-additional-full.csv', key_prefix=prefix)

In [8]:
from sagemaker.sklearn.processing import SKLearnProcessor

sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.m5.xlarge',
                                     instance_count=1)

In [9]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

sklearn_processor.run(code='preprocessing.py',
                      inputs=[ProcessingInput(
                        source=input_data,
                        destination='/opt/ml/processing/input')],
                      outputs=[ProcessingOutput(output_name='train_data',
                                                source='/opt/ml/processing/train'),
                               ProcessingOutput(output_name='test_data',
                                                source='/opt/ml/processing/test')],
                      arguments=['--train-test-split-ratio', '0.2']
                     )


Job Name:  sagemaker-scikit-learn-2021-09-25-16-27-01-743
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-203856535362/sagemaker/DEMO-smprocessing/input/bank-additional-full.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'train_data', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/train_data', 'LocalPath': '/opt/ml/processing/train', 'S3UploadMode': '

In [10]:
preprocessing_job_description = sklearn_processor.jobs[-1].describe()

output_config = preprocessing_job_description['ProcessingOutputConfig']
for output in output_config['Outputs']:
    print(output['S3Output']['S3Uri'])
    

s3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/train_data
s3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/test_data


In [11]:
!aws s3 cp s3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/train_data/train_features.csv .
!aws s3 cp s3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/train_data/train_labels.csv .    

/bin/bash: aws: command not found


In [ ]:
s3://sagemaker-us-east-1-203856535362/sagemaker-scikit-learn-2021-09-25-16-27-01-743/output/train_data/train_data/train_features.csv

In [18]:
cord('’')

NameError: name 'cord' is not defined